# Binary Classification using Convolutional Neural Networks

An investigation into the effects that image augmentation has on the accuracy and loss of Convolutional Neural Networks. *This work was completed as part of dissertation project for Bachelor of Science (Honours) in Computer Science with specialism in Artificial Intelligence.*

---

## Import Packages
Import all the necessary packages for the project to run.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
import datetime
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os


## Control Variables
The below variables are used to control various elements of the network model. They have been concentrated in a single area to make experimental variation easier to achieve, they act as a "control panel" for the operation of the model.